In [1]:
from transformers import AutoModelForSequenceClassification

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer

In [3]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384", num_labels=6000)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/Multilingual-MiniLM-L12-H384', fast=True)

In [5]:
model.cuda().eval()
inputs = tokenizer("Classify query: dildo", return_tensors="pt", 
    padding='max_length', truncation=True, max_length=50).to('cuda')

In [6]:
%%timeit 
inputs = tokenizer("Classify query: dildo", return_tensors="pt", 
    padding='max_length', truncation=True, max_length=50).to('cuda')

154 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
%%timeit
model(**inputs)

12.2 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
import torch._dynamo as torchdynamo
torchdynamo.config.cache_size_limit = 512

In [9]:
model.forward2 = torchdynamo.optimize("inductor")(model.forward)

In [10]:
# warm up
model.forward2(**inputs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0331, -0.0067,  0.0188,  ..., -0.0057,  0.0743, -0.0086]],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>), hidden_states=None, attentions=None)

In [11]:
%%timeit
model.forward2(**inputs)

2.14 ms ± 5.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
inputs2 = tokenizer("Classify query: dildo dildo dildo dildo", return_tensors="pt", 
    padding='max_length', truncation=True, max_length=50).to('cuda')

In [13]:
%%timeit
model.forward2(**inputs2)

2.17 ms ± 5.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
